In [6]:
from google.colab import files

uploaded = files.upload()

Saving stc TV Data Set_T3.xlsb to stc TV Data Set_T3.xlsb


In [9]:
!pip install pyxlsb
import pandas as pd
from pyxlsb import open_workbook

# Read xlsb file using pyxlsb
with open_workbook('stc TV Data Set_T3.xlsb') as wb:
    with wb.get_sheet(1) as sheet:
        data = []
        for row in sheet.rows():
            data.append([item.v for item in row])

# Convert to DataFrame
columns = data[0]
rows = data[1:]
df = pd.DataFrame(rows, columns=columns)

# Check the data
print(df.head())
print(df.shape)


   None user_id_maped          program_name  rating    date_ program_genre
0   0.0         26138            100 treets     1.0  42882.0         Drama
1   1.0          7946                 Moana     1.0  42876.0     Animation
2   2.0          7418  The Mermaid Princess     1.0  42957.0     Animation
3   3.0         19307  The Mermaid Princess     2.0  42942.0     Animation
4   4.0         15860             Churchill     2.0  42923.0     Biography
(1048575, 6)


In [10]:
# Drop rows with missing data
df.dropna(subset=['user_id_maped', 'program_name', 'rating'], inplace=True)

# Convert data types
df['user_id_maped'] = df['user_id_maped'].astype(int)
df['program_name'] = df['program_name'].astype(str)
df['rating'] = df['rating'].astype(int)


In [11]:
user_item_matrix = df.pivot_table(index='user_id_maped', columns='program_name', values='rating')
user_item_matrix.fillna(0, inplace=True)


In [12]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute similarity between items
item_similarity = cosine_similarity(user_item_matrix.T)
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)


In [13]:
def recommend_movies(movie_name, top_n=5):
    if movie_name not in item_similarity_df:
        return f"Movie '{movie_name}' not found in data."

    similar_scores = item_similarity_df[movie_name].sort_values(ascending=False)
    recommendations = similar_scores.iloc[1:top_n+1].index.tolist()
    return recommendations


In [14]:
print(recommend_movies("Moana", top_n=5))


['Trolls', "Surf's Up : WaveMania", 'The Mermaid Princess', 'The Boss Baby', 'The Jetsons & WWE: Robo-WrestleMania!']
